In [1]:
import os
import time
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf
# import tensorflow_hub as hub
import tensorflow.keras
import tensorflow_hub as hub
import tensorflow_text as text
from tensorflow.keras import backend as K

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import metrics as kmetrics
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

# Bert
import os
import transformers
from transformers import *

#scikitlearn & others

import tokenization
import pickle
import numpy as np
import sklearn.metrics as met
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier as RF

os.environ["TF_KERAS"]='1'

In [2]:
print(tf.__version__)
print(tensorflow.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(tf.test.is_built_with_cuda())
#tf.debugging.set_log_device_placement(True)

2.10.1
2.10.1
Num GPUs Available:  1
True


In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 768755850098580163
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10087301120
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7888724104429196045
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:02:00.0, compute capability: 6.1"
xla_global_id: 416903419
]


In [4]:
# verify GPU availability
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# SETUP data

In [5]:
path = 'D:/projet5/'
path_res = path +'results/'

In [6]:
with open(path + 'cleanDataprepSmote3.pickle','rb') as f:
    data = pickle.load(f)

In [7]:
data.shape

(51904, 20)

In [8]:
data.dtypes

Title                        object
Body                         object
Tags                         object
text                         object
nb_char                       int64
tokens                       object
tokens_title                 object
nb_tokens                     int64
tokens_unique                object
nb_tokens_unique              int64
tokens_WSW                   object
tokens_title_WSW             object
nb_tokens_WSW                 int64
tokens_lemma                 object
len_tokens_lemma              int64
Tags_list                    object
len _Tags_list                int64
filtered_tags                object
filtered_tags_reduced        object
len_filtered_tags_reduced     int64
dtype: object

In [9]:
data.head()

,Title,Body,Tags,text,nb_char,tokens,tokens_title,nb_tokens,tokens_unique,nb_tokens_unique,tokens_WSW,tokens_title_WSW,nb_tokens_WSW,tokens_lemma,len_tokens_lemma,Tags_list,len _Tags_list,filtered_tags,filtered_tags_reduced,len_filtered_tags_reduced
0,transpose/unzip function (inverse of zip)?,<p>I have a list of 2-item tuples and I'd like...,<python><list><matrix><transpose>,i have a list of 2-item tuples and i'd like to...,350,"[i, have, a, list, of, 2, item, tuples, and, i...","[transpose, unzip, function, inverse, of, zip]",70,"(c, example, 4, to, 1, become, tuple, 2, first...",42,"[list, item, tuples, like, convert, lists, fir...","[transpose, unzip, function, inverse, zip]",28,"[list, item, tuple, like, convert, list, first...",28,"[list, matrix, transpose, python]",4,"[list, python]",tag0,1
1,detecting audio silence in wav files using c#,<p>I'm tasked with building a .NET client app ...,<c#><.net><audio>,i'm tasked with building a .net client app to ...,190,"[i, m, tasked, with, building, a, net, client,...","[detecting, audio, silence, in, wav, files, us...",36,"(app, apis, to, silence, libraries, m, a, wav,...",30,"[tasked, building, net, client, app, detect, s...","[detecting, audio, silence, wav, files, using,...",17,"[task, build, net, client, app, detect, silenc...",17,"[.net, audio, c#]",3,"[.net, c#]",tag1,1
2,how to request a random row in sql?,<p>How can I request a random row (or as close...,<sql><random>,how can i request a random row (or as close to...,89,"[how, can, i, request, a, random, row, or, as,...","[how, to, request, a, random, row, in, sql]",19,"(possible, or, a, request, row, to, how, truly...",17,"[request, random, row, close, truly, random, p...","[request, random, row, sql]",9,"[request, random, row, close, truly, random, p...",9,"[random, sql]",2,[sql],tag108,1
4,detecting current iphone input language,"<p>Does anybody knows, can I get the current i...",<iphone><keyboard>,"does anybody knows, can i get the current inpu...",166,"[does, anybody, knows, can, i, get, the, curre...","[detecting, current, iphone, input, language]",28,"(knows, notification, also, layout, a, when, a...",23,"[anybody, knows, get, current, input, language...","[detecting, current, iphone, input, language]",16,"[anybody, know, get, current, input, language,...",16,"[iphone, keyboard]",2,[iphone],tag11,1
6,how can i use homebrew to install both python ...,<p>I need to be able to switch back and forth ...,<python><homebrew>,i need to be able to switch back and forth bet...,204,"[i, need, to, be, able, to, switch, back, and,...","[how, can, i, use, homebrew, to, install, both...",44,"(between, to, forth, using, 2, need, trouble, ...",34,"[need, able, switch, back, forth, python, usin...","[use, homebrew, install, python, mac]",16,"[need, able, switch, back, forth, python, use,...",16,"[homebrew, python]",2,[python],tag0,1


## shared functions

In [10]:
def wordFrequency(data):
    ''' take pandas series and count words retun the occurences of the elements'''
    freqTotal = {}
    for text in data:
        freq = nltk.Counter(text)
        for word, count in freq.items():
            if word in freqTotal.keys():
                freqTotal[word] += count
            else:
                freqTotal[word] = count
    return freqTotal

def splitData(df,XCol,Ycol,ratioToKeep=1):
    """ split data into two sets X and Y and then split them into  trainning and validation sets 
    the total will be 4 sets. the function returns pandas series or pandas dataframe types"""# add sample 
    df = df.sample(frac=ratioToKeep)
    print(f'data used shape: {df.shape}')
    X=df.loc[:,XCol]
    y=df.loc[:,Ycol]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    print(f'X_train types:\t{type(X_train)}')
    print(f'X_test types:\t{type(X_test)}')
    print(f'Y_test types:\t{type(y_test)}')
    print(f'Y_train types:\t{type(y_train)}')
    return X_train, X_test, y_train, y_test

def reduceData (Xtrain, Ytrain, ratioToKeep=0.2):
    size = Xtrain.shape[0]
    nbRowsToKeep = int(np.rint(size*ratioToKeep))
    indRowToKeep = np.random.choice(size, nbRowsToKeep)
    Xtrain = Xtrain[indRowToKeep,:]
    Ytrain = Ytrain[indRowToKeep,:]
    return (Xtrain,Ytrain)

In [11]:
def BOW(Xtrain, Xval):
    """ set the BOW method for X train and validation sets."""
    Xtrain = Xtrain.apply(lambda x : ' '.join(x))
    Xval = Xval.apply(lambda x : ' '.join(x))
    XtrainCorpus = [x for x in Xtrain]
    XvalCorpus = [x for x in Xval]
    vectorizer = CountVectorizer()
    Xtrain = vectorizer.fit_transform(XtrainCorpus)
    print('-'*15)
    print(f"Xtrain vectorized shape: {Xtrain.shape}\n")
    Xval = vectorizer.transform(XvalCorpus)
    print(f"Xval vectorized shape: {Xval.shape}\n")
    return (Xtrain,Xval)

def Binarize(Ytrain,Yval):
    """  binarize The tags for Y train and validation sets. """
    mlb = MultiLabelBinarizer()
    mlb.fit([Ytrain]) # if Y is not list of list, mlb.fit(['sci-fi', 'thriller', 'comedy']) -> array(['-', 'c', 'd', 'e', 'f', 'h', 'i', 'l', 'm', 'o', 'r', 's', 't','y'], dtype=object)
    for i in Ytrain.index:
        Ytrain.loc[i] = mlb.transform([[Ytrain.loc[i]]])[0]
    for i in Yval.index:
        Yval.loc[i] = mlb.transform([[Yval.loc[i]]])[0]
    labels = mlb.classes_
    print('-'*15)
    print(f"Ytrain vectorized shape: {Ytrain.shape}\n")
    print(f"Yval vectorized shape: {Yval.shape}\n")
    print(f"classes : {mlb.classes_}")
    return (Ytrain,Yval, labels)

def plotRoc():
    # Compute ROC curve and ROC area for each class
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(len(labels)):
        fpr[i], tpr[i], _ = roc_curve(y_val[:, i], y_pred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    # Compute micro-average ROC curve and ROC area
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
    roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
    plt.figure()
    lw = 2
    plt.plot(
        fpr[2],
        tpr[2],
        color="darkorange",
        lw=lw,
        label="ROC curve (area = %0.2f)" % roc_auc[2],
    )
    plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("Receiver operating characteristic example")
    plt.legend(loc="lower right")
    plt.show()





def clfScore(model,Xval, Yval, labels, fileName,PATH=path_res):                           #  correct position of the labels ???
    y_pred = model.predict(Xval)
    F1score=met.f1_score(y_pred, Yval, average=None)
    preciScore=met.precision_score(y_pred, Yval, average=None)
    recallScore=met.recall_score(y_pred, Yval, average=None)
    print(f'{"Tags":25}\t\t F-score\t\t precision\t\t recall\n')
    with open(PATH + fileName,'w') as f:
        for ind,name in enumerate(labels):
            f.write(f'{name:25}\t\t {F1score[ind]:4f}\t\t {preciScore[ind]:4f}\t\t {recallScore[ind]:4f}\n')
            print(f'{name:25}\t\t {F1score[ind]:4f}\t\t {preciScore[ind]:4f}\t\t {recallScore[ind]:4f}')
    #plotRoc(labels,y_pred,y_val)
    
        

    
    

# USE

In [12]:
encoder_url = 'https://tfhub.dev/google/universal-sentence-encoder/4'

In [13]:
encoderModel = hub.KerasLayer(encoder_url)

In [14]:
def feature_USE_fct(sentences, b_size) :
    batch_size = b_size
    time1 = time.time()
    divisor = len(sentences)//batch_size
    remainder = len(sentences)%batch_size
    for step in range(divisor) :
        idx = step*batch_size
        feat = encoderModel(sentences[idx:idx+batch_size])

        if step ==0 :
            features = feat
        else :
            features = np.concatenate((features,feat))
        if (step+1) == divisor and remainder !=0:
            feat = encoderModel(sentences[divisor*batch_size:])
            features = np.concatenate((features,feat))

    time2 = np.round(time.time() - time1,0)
    return features

In [15]:
X_train, X_val, y_train, y_val = splitData(data,XCol='tokens_WSW',Ycol='filtered_tags_reduced',ratioToKeep=0.5) # too much time without sampling
print(f"y_train shape: {y_train.shape}\n")
print(f"y_val shape: {y_val.shape}\n")
print(f"X_train shape: {X_train.shape}\n")
print(f"X_val shape: {X_val.shape}\n")

data used shape: (25952, 20)
X_train types:	<class 'pandas.core.series.Series'>
X_test types:	<class 'pandas.core.series.Series'>
Y_test types:	<class 'pandas.core.series.Series'>
Y_train types:	<class 'pandas.core.series.Series'>
y_train shape: (17387,)

y_val shape: (8565,)

X_train shape: (17387,)

X_val shape: (8565,)



In [16]:
X_train = X_train.apply(lambda x: ' '.join(x).strip())
X_val = X_val.apply(lambda x: ' '.join(x).strip())
X_val.reset_index(inplace=True,drop=True)
X_train.reset_index(inplace=True,drop=True)
y_train.reset_index(inplace=True,drop=True)
y_val.reset_index(inplace=True,drop=True)

In [17]:
batch_size = 10
X_train = X_train.to_list()
X_val = X_val.to_list()
X_train = feature_USE_fct(X_train, batch_size)
X_val = feature_USE_fct(X_val, batch_size)


In [18]:
#y_train = ['nice','bad','nice']
#y_val = ['nice','bad']

y_train, y_val, labels = Binarize(y_train,y_val)
y_train = [ i for i in y_train]
y_val = [ i for i in y_val]
y_train = np.asarray(y_train)
y_val = np.asarray(y_val)

print(f"y_train vectorized shape: {y_train.shape}\n")
print(f"y_val vectorized shape: {y_val.shape}\n")

oversample = SMOTE(random_state=42)
X_train, y_train = oversample.fit_resample(X_train, y_train) # SMOTE
X_train, y_train = shuffle(X_train,y_train, random_state=1)
print(f"---- After SMOTE ----")
print(f"y_train vectorized shape: {y_train.shape}\n")
print(f"X_train vectorized shape: {X_train.shape}\n")

print(f"---- Reduce size training set ----")
X_train, y_train = reduceData(X_train ,y_train,ratioToKeep=0.5) #0.05

print(f"y_train vectorized shape: {y_train.shape}\n")
print(f"y_val vectorized shape: {y_val.shape}\n")
print(f"X_train vectorized shape: {X_train.shape}\n")
print(f"X_val vectorized shape: {X_val.shape}\n")

---------------
Ytrain vectorized shape: (17387,)

Yval vectorized shape: (8565,)

classes : ['tag0' 'tag1' 'tag10' 'tag108' 'tag11' 'tag12' 'tag13' 'tag14' 'tag15'
 'tag16' 'tag17' 'tag18' 'tag19' 'tag2' 'tag20' 'tag21' 'tag22' 'tag23'
 'tag24' 'tag25' 'tag26' 'tag27' 'tag28' 'tag29' 'tag3' 'tag30' 'tag31'
 'tag32' 'tag33' 'tag34' 'tag35' 'tag36' 'tag37' 'tag38' 'tag4' 'tag5'
 'tag6' 'tag7' 'tag8' 'tag9']
y_train vectorized shape: (17387, 40)

y_val vectorized shape: (8565, 40)

---- After SMOTE ----
y_train vectorized shape: (71240, 40)

X_train vectorized shape: (71240, 512)

---- Reduce size training set ----
y_train vectorized shape: (35620, 40)

y_val vectorized shape: (8565, 40)

X_train vectorized shape: (35620, 512)

X_val vectorized shape: (8565, 512)



###  logistic regression

In [19]:
#y_train, y_val, labels = Binarize(y_train,y_val)

log_clf = OneVsRestClassifier(LogisticRegression(random_state=0,
                                                 max_iter=10000,
                                                 verbose=0))

model = Pipeline(steps=[("scaler" , StandardScaler()),('estimator',log_clf)])
#raise Exception("trop long") 
# hyperparameters
param_grid = [{'estimator__estimator__C': [0.1, 0.5, 1., 5, 10]}] # lambda regularization

# cross-validation
clf = GridSearchCV(model,
                   param_grid,
                   scoring='f1_macro', #F1 = 2 * (precision * recall) / (precision + recall)
                   n_jobs=-1,
                   verbose=0,
                   cv=5,
                   return_train_score=True)
clf.fit(X_train, y_train)
log_clf_best_param = clf.best_params_['estimator__estimator__C']
print('Meilleurs hyper-paramètres :')
print(clf.best_params_)

Meilleurs hyper-paramètres :
{'estimator__estimator__C': 5}


In [20]:
log_clf = OneVsRestClassifier(LogisticRegression(random_state=42,
                                                 max_iter=10000,
                                                 verbose=0,
                                                C=log_clf_best_param))
model = Pipeline(steps=[("scaler" , StandardScaler()),('estimator',log_clf)])
model.fit(X_train, y_train)
clfScore(model, X_val,y_val, labels,'USE_logistic_reduced_without_empty')

Tags                     		 F-score		 precision		 recall

tag0                     		 0.564356		 0.457045		 0.737523
tag1                     		 0.364677		 0.257803		 0.622905
tag10                    		 0.374728		 0.436548		 0.328244
tag108                   		 0.453721		 0.498008		 0.416667
tag11                    		 0.291829		 0.365854		 0.242718
tag12                    		 0.808140		 0.817647		 0.798851
tag13                    		 0.252588		 0.321053		 0.208191
tag14                    		 0.340314		 0.445205		 0.275424
tag15                    		 0.262626		 0.376812		 0.201550
tag16                    		 0.203310		 0.328244		 0.147260
tag17                    		 0.204489		 0.305970		 0.153558
tag18                    		 0.176471		 0.294643		 0.125954
tag19                    		 0.310231		 0.419643		 0.246073
tag2                     		 0.523810		 0.447025		 0.632444
tag20                    		 0.213873		 0.308333		 0.163717
tag21                    		 0.279070		 0.342857		 0.23529

## Random Forest

In [21]:
RF_clf = OneVsRestClassifier(RF(min_samples_split=5))
model = Pipeline(steps=[("scaler" , StandardScaler(with_mean=False)),('estimator',RF_clf)])
#raise Exception("trop long") 
# hyperparameters
param_grid = [{'estimator__estimator__n_estimators': [10,20,30],'estimator__estimator__max_depth' : [5,8,10,15]}]

# cross-validation
clf = GridSearchCV(model,
                   param_grid,
                   scoring='f1_macro', #F1 = 2 * (precision * recall) / (precision + recall)
                   n_jobs=-1,
                   verbose=0,
                   cv=5,
                   return_train_score=True)
clf.fit(X_train, y_train)
RF_clf_best_param1 = clf.best_params_['estimator__estimator__n_estimators']
RF_clf_best_param2 = clf.best_params_['estimator__estimator__max_depth']
print('Meilleurs hyper-paramètres :')
print(clf.best_params_)

Meilleurs hyper-paramètres :
{'estimator__estimator__max_depth': 15, 'estimator__estimator__n_estimators': 30}


In [22]:
RF_clf = OneVsRestClassifier(RF(n_estimators=RF_clf_best_param1,
                                                  max_depth=RF_clf_best_param2,
                                                   min_samples_split=5))
model = Pipeline(steps=[("scaler" , StandardScaler(with_mean=False)),('estimator',RF_clf)])
model.fit(X_train, y_train)
clfScore(model, X_val,y_val, labels,'USE_RF_reduced_without_empty')

Tags                     		 F-score		 precision		 recall

tag0                     		 0.223541		 0.129439		 0.818841
tag1                     		 0.031180		 0.016185		 0.424242
tag10                    		 0.335404		 0.274112		 0.432000
tag108                   		 0.360563		 0.254980		 0.615385
tag11                    		 0.073801		 0.048780		 0.151515
tag12                    		 0.724739		 0.611765		 0.888889
tag13                    		 0.041152		 0.026316		 0.094340
tag14                    		 0.259615		 0.184932		 0.435484
tag15                    		 0.048193		 0.028986		 0.142857
tag16                    		 0.148571		 0.099237		 0.295455
tag17                    		 0.105263		 0.067164		 0.243243
tag18                    		 0.083832		 0.062500		 0.127273
tag19                    		 0.184397		 0.116071		 0.448276
tag2                     		 0.257143		 0.156749		 0.715232
tag20                    		 0.162500		 0.108333		 0.325000
tag21                    		 0.117647		 0.076190		 0.25806

D:\DevPackages\lib\site-packages\joblib\_memmapping_reducer.py:607: UserWarning: Failed to delete temporary folder: C:\Users\thomas\AppData\Local\Temp\joblib_memmapping_folder_2160_a0861911983344f7a3b23081f38cc110_c4c0d6b4136740e497931b5481e7b670
  warnings.warn("Failed to delete temporary folder: {}"
D:\DevPackages\lib\site-packages\joblib\_memmapping_reducer.py:607: UserWarning: Failed to delete temporary folder: C:\Users\thomas\AppData\Local\Temp\joblib_memmapping_folder_2160_ece8b610232544889d2de0e3a92b8c14_f746d67d3f1a4876b62ece5d88f64c33
  warnings.warn("Failed to delete temporary folder: {}"
